# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Mar 1 2023 2:36PM,258473,10,PRF-53364-BO,Bio-PRF,Released
1,Mar 1 2023 2:36PM,258473,10,PRF-54297-BO,Bio-PRF,Released
2,Mar 1 2023 2:36PM,258473,10,PRF-46304-BO,Bio-PRF,Released
3,Mar 1 2023 2:36PM,258473,10,PRF-46360-BO,Bio-PRF,Released
4,Mar 1 2023 2:36PM,258473,10,PRF-47210-BO,Bio-PRF,Released
5,Mar 1 2023 2:36PM,258473,10,PRF-53385-BO,Bio-PRF,Released
6,Mar 1 2023 2:35PM,258472,10,9217717,Eywa Pharma Inc.,Released
7,Mar 1 2023 2:35PM,258472,10,9220343,Eywa Pharma Inc.,Released
8,Mar 1 2023 2:35PM,258472,10,9220345,Eywa Pharma Inc.,Released
9,Mar 1 2023 2:35PM,258472,10,9220346,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,258462,Released,9
16,258467,Released,17
17,258470,Released,1
18,258472,Released,5
19,258473,Released,6


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
258462,NaN,9.0
258467,NaN,17.0
258470,NaN,1.0
258472,NaN,5.0
258473,NaN,6.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
258409,2.0,0.0
258417,0.0,1.0
258420,0.0,2.0
258421,1.0,0.0
258434,0.0,34.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
258409,2,0
258417,0,1
258420,0,2
258421,1,0
258434,0,34


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,258409,2,0
1,258417,0,1
2,258420,0,2
3,258421,1,0
4,258434,0,34


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,258409,2,
1,258417,,1
2,258420,,2
3,258421,1,
4,258434,,34


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Mar 1 2023 2:36PM,258473,10,Bio-PRF
6,Mar 1 2023 2:35PM,258472,10,Eywa Pharma Inc.
11,Mar 1 2023 2:06PM,258470,10,Yusen – 3D Matrix
12,Mar 1 2023 1:33PM,258467,10,ISDIN Corporation
29,Mar 1 2023 1:12PM,258462,12,Hush Hush
38,Mar 1 2023 12:57PM,258459,10,"ClearSpec, LLC"
39,Mar 1 2023 11:18AM,258454,10,Methapharm-G
41,Mar 1 2023 11:18AM,258454,10,"Methapharm, Inc."
45,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc."
46,Mar 1 2023 10:53AM,258448,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Mar 1 2023 2:36PM,258473,10,Bio-PRF,,6
1,Mar 1 2023 2:35PM,258472,10,Eywa Pharma Inc.,,5
2,Mar 1 2023 2:06PM,258470,10,Yusen – 3D Matrix,,1
3,Mar 1 2023 1:33PM,258467,10,ISDIN Corporation,,17
4,Mar 1 2023 1:12PM,258462,12,Hush Hush,,9
5,Mar 1 2023 12:57PM,258459,10,"ClearSpec, LLC",,1
6,Mar 1 2023 11:18AM,258454,10,Methapharm-G,5,1
7,Mar 1 2023 11:18AM,258454,10,"Methapharm, Inc.",5,1
8,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc.",1,
9,Mar 1 2023 10:53AM,258448,22,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 1 2023 2:36PM,258473,10,Bio-PRF,6,
1,Mar 1 2023 2:35PM,258472,10,Eywa Pharma Inc.,5,
2,Mar 1 2023 2:06PM,258470,10,Yusen – 3D Matrix,1,
3,Mar 1 2023 1:33PM,258467,10,ISDIN Corporation,17,
4,Mar 1 2023 1:12PM,258462,12,Hush Hush,9,
5,Mar 1 2023 12:57PM,258459,10,"ClearSpec, LLC",1,
6,Mar 1 2023 11:18AM,258454,10,Methapharm-G,1,5
7,Mar 1 2023 11:18AM,258454,10,"Methapharm, Inc.",1,5
8,Mar 1 2023 11:00AM,258451,10,"Citieffe, Inc.",,1
9,Mar 1 2023 10:53AM,258448,22,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 1 2023 2:36PM,258473,10,Bio-PRF,6,
1,Mar 1 2023 2:35PM,258472,10,Eywa Pharma Inc.,5,
2,Mar 1 2023 2:06PM,258470,10,Yusen – 3D Matrix,1,
3,Mar 1 2023 1:33PM,258467,10,ISDIN Corporation,17,
4,Mar 1 2023 1:12PM,258462,12,Hush Hush,9,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Mar 1 2023 2:36PM,258473,10,Bio-PRF,6.0,NaN
1,Mar 1 2023 2:35PM,258472,10,Eywa Pharma Inc.,5.0,NaN
2,Mar 1 2023 2:06PM,258470,10,Yusen – 3D Matrix,1.0,NaN
3,Mar 1 2023 1:33PM,258467,10,ISDIN Corporation,17.0,NaN
4,Mar 1 2023 1:12PM,258462,12,Hush Hush,9.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Mar 1 2023 2:36PM,258473,10,Bio-PRF,6.0,0.0
1,Mar 1 2023 2:35PM,258472,10,Eywa Pharma Inc.,5.0,0.0
2,Mar 1 2023 2:06PM,258470,10,Yusen – 3D Matrix,1.0,0.0
3,Mar 1 2023 1:33PM,258467,10,ISDIN Corporation,17.0,0.0
4,Mar 1 2023 1:12PM,258462,12,Hush Hush,9.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,3101452,146.0,11.0
12,516883,9.0,1.0
15,516849,1.0,2.0
19,516861,3.0,0.0
22,516865,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,3101452,146.0,11.0
1,12,516883,9.0,1.0
2,15,516849,1.0,2.0
3,19,516861,3.0,0.0
4,22,516865,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,146.0,11.0
1,12,9.0,1.0
2,15,1.0,2.0
3,19,3.0,0.0
4,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,146.0
1,12,Released,9.0
2,15,Released,1.0
3,19,Released,3.0
4,22,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,22
Status,,,,,
Executing,11.0,1.0,2.0,0.0,0.0
Released,146.0,9.0,1.0,3.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,22
0,Executing,11.0,1.0,2.0,0.0,0.0
1,Released,146.0,9.0,1.0,3.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,22
0,Executing,11.0,1.0,2.0,0.0,0.0
1,Released,146.0,9.0,1.0,3.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()